<a href="https://colab.research.google.com/github/GiliardGodoi/tj-datasets/blob/main/notebooks/preprocessamento/2024_05_16_Voto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive, userdata
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install "tjdatasets @ git+https://github.com/GiliardGodoi/tj-datasets@may-24"

  Cloning https://github.com/GiliardGodoi/tj-datasets (to revision may-24) to /tmp/pip-install-tie9c0te/tjdatasets_f480362a28464d42a6902d67b15ff9fa
  Running command git clone --filter=blob:none --quiet https://github.com/GiliardGodoi/tj-datasets /tmp/pip-install-tie9c0te/tjdatasets_f480362a28464d42a6902d67b15ff9fa
  Running command git checkout -b may-24 --track origin/may-24
  Switched to a new branch 'may-24'
  Branch 'may-24' set up to track remote branch 'may-24' from 'origin'.
  Resolved https://github.com/GiliardGodoi/tj-datasets to commit d5f1808ce70abcb0a7fb4768abcc5907d1a02836
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tjdatasets: filename=tjdatasets-0.0.4-py3-none-any.whl size=18533 sha256=d2807fd6874c998659bab4448f4d01abda207631397b1fe58bf99506cf8e073f
  Stored in directory: /tmp/pip-ephem-wheel-cache-n294999s/wheels/46/7

In [ ]:
import pandas as pd

from pathlib import Path

In [ ]:
from tjdatasets.pipe import (
    ProcessamentoFundamentacao,
    ProcessamentoVotoRelator,
    PreProcessamentoNormalizado
)

In [ ]:
BASE_FOLDER = Path('/content/drive/Shareddrives/Projeto_TJSP/Projeto Análise de Precedentes/Datasets/base_set_2023/RAW')
OUTPUT_FOLDER = Path('/content/drive/Shareddrives/Projeto_TJSP_Datasets/Datasets/base_abril_2024/PREPROCESSED')

base_33k_filepath = BASE_FOLDER / "acordaos_principais_33k.csv"
base_40k_filepath = BASE_FOLDER / "acordaos_principais_40k.csv"

assert base_33k_filepath.exists()
assert base_40k_filepath.exists()
assert OUTPUT_FOLDER.exists()

In [ ]:
fragmentador_fundamentacao = ProcessamentoFundamentacao(column_text='conteudo')
fragmentador_voto_relator  = ProcessamentoVotoRelator(column_text='conteudo')

normalizador = PreProcessamentoNormalizado()

for filepath in [base_33k_filepath, base_40k_filepath]:
    print('Processando...', filepath.name)

    df = pd.read_csv(filepath, encoding='utf8')
    print('Lido...', df.shape)

    texto_fundamentacao = fragmentador_fundamentacao.transform(df)
    texto_normalizado = normalizador.transform(texto_fundamentacao)

    df['formatado_fundamentacao'] = texto_fundamentacao
    df['formatado_fundamentacao_normalizado'] = texto_normalizado

    texto_voto_relator = fragmentador_voto_relator.transform(df)
    texto_voto_relator_normalizado = normalizador.transform(texto_voto_relator)

    df['formatado_voto_relator'] = texto_voto_relator
    df['formatado_voto_relator_normalizado'] = texto_voto_relator_normalizado

    dest = OUTPUT_FOLDER / f"{filepath.stem}_FundamentacaoDecisao_VotoRelator.parquet.gzip"
    print("Salvando em...", dest)
    df.to_parquet(dest, compression='gzip')

Processando... acordaos_principais_33k.csv
Lido... (33016, 5)
Salvando em... /content/drive/Shareddrives/Projeto_TJSP_Datasets/Datasets/base_abril_2024/PREPROCESSED/acordaos_principais_33k_FundamentacaoDecisao_VotoRelator.parquet.gzip
Processando... acordaos_principais_40k.csv
Lido... (41374, 5)
Salvando em... /content/drive/Shareddrives/Projeto_TJSP_Datasets/Datasets/base_abril_2024/PREPROCESSED/acordaos_principais_40k_FundamentacaoDecisao_VotoRelator.parquet.gzip


In [ ]:
df = pd.read_parquet(OUTPUT_FOLDER / "acordaos_principais_33k_FundamentacaoDecisao_VotoRelator.parquet.gzip")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33016 entries, 0 to 33015
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   numero_processo                      33016 non-null  object
 1   id_documento                         33016 non-null  object
 2   conteudo                             33016 non-null  object
 3   data_hora_documento                  33016 non-null  object
 4   codigos_movimentos_temas             33016 non-null  object
 5   formatado_fundamentacao              33016 non-null  object
 6   formatado_fundamentacao_normalizado  33016 non-null  object
 7   formatado_voto_relator               33016 non-null  object
 8   formatado_voto_relator_normalizado   33016 non-null  object
dtypes: object(9)
memory usage: 2.3+ MB


In [ ]:
df.sample(5)

,numero_processo,id_documento,conteudo,data_hora_documento,codigos_movimentos_temas,formatado_fundamentacao,formatado_fundamentacao_normalizado,formatado_voto_relator,formatado_voto_relator_normalizado
18027,10002028520168260458,124021296 - 0,PODER JUDICIÁRIO\nTRIBUNAL DE JUSTIÇA DO ESTAD...,2022-09-12 00:00:00,85258;85351;85524;85609,"de plano, e caso de diferimento do recolhimen...",plano diferimento recolhimento custas razao na...,assinatura eletronica poder judiciario tribuna...,assinatura eletronica judiciario justica sao_p...
14158,21546726820198260000,84693397 - 0,PODER JUDICIÁRIO\nTRIBUNAL DE JUSTIÇA DO ESTAD...,2020-06-22 00:00:00,80718,"o recurso comporta provimento. na hipotese, n...",comporta provimento hipotese embargos_de_decla...,assinatura eletronica poder judiciario tribun...,assinatura eletronica judiciario justica sao_p...
31512,22233836220188260000,102267212 - 0,PODER JUDICIÁRIO\nTRIBUNAL DE JUSTIÇA DO ESTAD...,2021-06-12 00:00:00,85166;85258;85351;85510;85524;85609,no que se refere a prefacial de ilegitimidade...,refere prefacial ilegitimidade ativa poupador ...,assinatura eletronica poder judiciario tribuna...,assinatura eletronica judiciario justica sao_p...
10538,21550701520198260000,74878220 - 0,em -— PODER JUDICIÁRIO\num TRIBUNAL DE JUSTIÇA...,2019-10-18,85714,assinatura eletronica em - poder judiciario um...,assinatura eletronica judiciario justica sao_p...,assinatura eletronica em - poder judiciario um...,assinatura eletronica judiciario justica sao_p...
2152,00015531620148260318,46571664 - 0,PODER JUDICIÁRIO\nTRIBUNAL DE JUSTIÇA DO ESTAD...,2017-11-24,80101;80554;84999;85609;85696,"o. cumpre observar, inicialmente, que os atos ...",cumpre observar inicialmente atos processuais ...,assinatura eletronica poder judiciario tribuna...,assinatura eletronica judiciario justica sao_p...


In [ ]:
df = pd.read_parquet(OUTPUT_FOLDER / "acordaos_principais_40k_FundamentacaoDecisao_VotoRelator.parquet.gzip")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41374 entries, 0 to 41373
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   numero_processo                      41374 non-null  object
 1   id_documento                         41374 non-null  object
 2   data_hora_documento                  41374 non-null  object
 3   conteudo                             41374 non-null  object
 4   codigos_movimentos_temas             41374 non-null  object
 5   formatado_fundamentacao              41374 non-null  object
 6   formatado_fundamentacao_normalizado  41374 non-null  object
 7   formatado_voto_relator               41374 non-null  object
 8   formatado_voto_relator_normalizado   41374 non-null  object
dtypes: object(9)
memory usage: 2.8+ MB


In [ ]:
df.sample(5)

,numero_processo,id_documento,data_hora_documento,conteudo,codigos_movimentos_temas,formatado_fundamentacao,formatado_fundamentacao_normalizado,formatado_voto_relator,formatado_voto_relator_normalizado
4016,10021426920188260279,71299939 - 0,2019-09-19 00:00:00,"TRIBUNAL DE JUSTIÇA\n\na, 5 = al ,\n== PODER J...",,"assinatura eletronica tribunal de justica a, 5...",assinatura eletronica justica _numero_ judicia...,"assinatura eletronica tribunal de justica a, 5...",assinatura eletronica justica _numero_ judicia...
22341,20367422920198260000,68108524 - 0,2019-05-28 00:00:00,PODER JUDICIÁRIO\nTRIBUNAL DE JUSTIÇA DO ESTAD...,,assinatura eletronica es pe poder judiciario m...,assinatura eletronica judiciario recurso_extra...,assinatura eletronica es pe poder judiciario m...,assinatura eletronica judiciario recurso_extra...
37413,21820471020208260000,87287859 - 0,2020-08-14 00:00:00,== PODER JUDICIÁRIO\nmio] TRIBUNAL DE JUSTIÇA ...,,assinatura eletronica es pe poder judiciario m...,assinatura eletronica judiciario justica sao_p...,assinatura eletronica es pe poder judiciario m...,assinatura eletronica judiciario justica sao_p...
33780,21459779120208260000,86069715 - 0,2020-07-16 00:00:00,TRIBUNAL DE JUSTIÇA\n\nEs Po PODER JUDICIÁRIO\...,,assinatura eletronica tribunal de justica es p...,assinatura eletronica justica judiciario sao_p...,assinatura eletronica tribunal de justica es p...,assinatura eletronica justica judiciario sao_p...
22335,20367061620218260000,98074219 - 0,2021-03-15 00:00:00,== PODER JUDICIÁRIO\nmm iDE TRIBUNAL DE JUSTIÇ...,,assinatura eletronica == poder judiciario mm e...,assinatura eletronica judiciario justica sao_p...,assinatura eletronica == poder judiciario mm e...,assinatura eletronica judiciario justica sao_p...
